In [30]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the original data
original_data = pd.read_csv('data/car-price.csv')

In [31]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np

def preprocess(data):
    
    # Replace '-' with NaN in 'Levy'
    data['Levy'].replace('-', np.nan, inplace=True)
    # Convert 'Levy' to numerical type
    data['Levy'] = pd.to_numeric(data['Levy'])  
    # Convert 'Mileage' to numerical type
    if data['Mileage'].dtype == 'object':
        data['Mileage'] = data['Mileage'].str.replace(' km', '')
    data['Mileage'] = pd.to_numeric(data['Mileage']) 


    # Extract 'Turbo' information and convert 'Engine volume' to numerical type
    data['Turbo'] = data['Engine volume'].apply(lambda x: 1 if isinstance(x, str) and 'Turbo' in x else 0)
    if data['Engine volume'].dtype == 'object':
        data['Engine volume'] = data['Engine volume'].str.replace(' Turbo', '')
    data['Engine volume'] = pd.to_numeric(data['Engine volume'])
    
    # Leather interior convert to boolean
    data['Leather interior'] = data['Leather interior'].map({'Yes': 1, 'No': 0})

    # Map the 'Doors' column
    doors_mapping = {
        '04-May': '4-5',
        '02-Mar': '2-3',
        '>5': '>5'
    }
    data['Doors'] = data['Doors'].map(doors_mapping)
    
    outlier_columns = ['Price','Levy', 'Mileage', 'Engine volume']

    # Remove outliers
    for col in outlier_columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
        
    top_manufacturers = data['Manufacturer'].value_counts().nlargest(10).index
    data['Manufacturer'] = data['Manufacturer'].apply(lambda x: x if x in top_manufacturers else 'Other')
    data['Manufacturer'] = data['Manufacturer'].str.title()
    top_categories = data['Category'].value_counts().nlargest(5).index
    data['Category'] = data['Category'].apply(lambda x: x if x in top_categories else 'Other')
    top_colors = data['Color'].value_counts().nlargest(5).index
    data['Color'] = data['Color'].apply(lambda x: x if x in top_colors else 'Other')
    top_fuel_types = data['Fuel type'].value_counts().nlargest(3).index
    data.loc[~data['Fuel type'].isin(top_fuel_types), 'Fuel type'] = 'Other'
    
    # Remove 'Model' and 'ID' columns
    data = data.drop(['Model', 'ID'], axis=1)
    data['Price'] = data['Price'].apply(lambda x: max(x,500))
    
#     cols = list(data.columns)
#     cols.remove('Price')
#     cols.append('Price')
#     data = data[cols]
    
#     rename_cols = {"Levy":"levy", "Manufacturer": "manufacturer",  'Prod. year': "year", 
#               'Category': 'category', 'Leather interior': "leather_interior",'Fuel type':'fuel_type',
#               'Engine volume':'engine_volume', 'Mileage': 'mileage', 'Cylinders': 'cylinders',
#               'Gear box type': 'gear_box_type', 'Drive wheels':'drive_wheels', 'Doors':'doors',
#               "Wheel":'wheel', 'Color':'color', 'Airbags': 'airbags', 'Turbo': "turbo", 'Price': 'price'}
    
#     data = data.rename(columns = rename_cols)
    
#     data.to_csv("data/data.csv")
    
#     return data
    
    # Separate features and target
    X = data.drop('Price', axis=1)
    y = data['Price']

    num_cols = X.select_dtypes(exclude="object").columns
    cat_cols = X.select_dtypes(include="object").columns
    
    imputer = SimpleImputer(strategy='median')
    for col in num_cols:
        data[col] = imputer.fit_transform(data[col].values.reshape(-1, 1))
        

#     # Apply StandardScaler to numerical columns and OneHotEncoder to categorical columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_cols),
            ('cat', OneHotEncoder(), cat_cols)
        ])
    X_transformed = preprocessor.fit_transform(X)
    
#     # Get the feature names after one-hot encoding
    feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(input_features=cat_cols)
    feature_names = np.concatenate([num_cols, feature_names])
    
    return X_transformed, y, feature_names


In [22]:
cleaned_data = preprocess(original_data)
cleaned_data

,levy,manufacturer,year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags,turbo,price
0,1399.0,Lexus,2010,Jeep,1,Hybrid,3.5,186005,6.0,Automatic,4x4,4-5,Left wheel,Silver,12,0,13328
1,1018.0,Chevrolet,2011,Jeep,0,Petrol,3.0,192000,6.0,Tiptronic,4x4,4-5,Left wheel,Black,8,0,16621
3,862.0,Ford,2011,Jeep,1,Hybrid,2.5,168966,4.0,Automatic,4x4,4-5,Left wheel,White,0,0,3607
4,446.0,Honda,2014,Hatchback,1,Petrol,1.3,91901,4.0,Automatic,Front,4-5,Left wheel,Silver,4,0,11726
5,891.0,Hyundai,2016,Jeep,1,Diesel,2.0,160931,4.0,Automatic,Front,4-5,Left wheel,White,4,0,39493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19231,1055.0,Mercedes-Benz,2013,Sedan,1,Diesel,3.5,107800,6.0,Automatic,Rear,4-5,Left wheel,Grey,12,0,5802
19233,831.0,Hyundai,2011,Sedan,1,Petrol,2.4,161600,4.0,Tiptronic,Front,4-5,Left wheel,Other,8,0,15681
19234,836.0,Hyundai,2010,Jeep,1,Diesel,2.0,116365,4.0,Automatic,Front,4-5,Left wheel,Grey,4,0,26108
19235,1288.0,Chevrolet,2007,Jeep,1,Diesel,2.0,51258,4.0,Automatic,Front,4-5,Left wheel,Black,4,0,5331


In [29]:
cleaned_data['mileage'].max()

336971

In [38]:
cleaned_data.dtypes

levy                float64
manufacturer         object
year                  int64
category             object
leather_interior      int64
fuel_type            object
engine_volume       float64
mileage               int64
cylinders           float64
gear_box_type        object
drive_wheels         object
doors                object
wheel                object
color                object
airbags               int64
turbo                 int64
price                 int64
dtype: object

In [41]:
for col in cleaned_data.columns:
    print(col)
    if cleaned_data[col].dtype == object:
        print(cleaned_data[col].unique())

levy
manufacturer
['Lexus' 'Chevrolet' 'Ford' 'Honda' 'Hyundai' 'Toyota' 'Mercedes-Benz'
 'Other' 'Bmw' 'Ssangyong' 'Kia']
year
category
['Jeep' 'Hatchback' 'Sedan' 'Other' 'Universal' 'Minivan']
leather_interior
fuel_type
['Hybrid' 'Petrol' 'Diesel' 'Other']
engine_volume
mileage
cylinders
gear_box_type
['Automatic' 'Tiptronic' 'Manual' 'Variator']
drive_wheels
['4x4' 'Front' 'Rear']
doors
['4-5' '>5' '2-3']
wheel
['Left wheel' 'Right-hand drive']
color
['Silver' 'Black' 'White' 'Grey' 'Blue' 'Other']
airbags
turbo
price


In [16]:
# cat_cols = cleaned_data.select_dtypes(include="object").columns

# for col in cat_cols:
#     print(col.lower(), list(cleaned_data[col].unique()))

In [32]:
# Assume that 'new_data' is a DataFrame containing your new data
X,y,features = preprocess(original_data)

In [33]:
len(features)

47

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the models
model_rf = RandomForestRegressor(random_state=42)
model_cb = CatBoostRegressor(random_seed=42, verbose=0)  # verbose=0 to turn off training output
model_xgb = XGBRegressor(random_state=42)

# Train the models
model_rf.fit(X_train, y_train)
model_cb.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)

# Use the models to make predictions on the testing data
y_pred_rf = model_rf.predict(X_test)
y_pred_cb = model_cb.predict(X_test)
y_pred_xgb = model_xgb.predict(X_test)

# Combine the predictions
y_pred_combined = (y_pred_rf + y_pred_cb + y_pred_xgb) / 3

# Calculate and print the root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred_combined))
print(f'Root Mean Squared Error: {rmse}')

# Calculate and print the R-squared value
r2 = r2_score(y_test, y_pred_combined)
print(f'R-squared: {r2}')

Root Mean Squared Error: 5149.536120084419
R-squared: 0.8180503724677747


In [ ]:
    # Get a list of all column names
#     cols = list(data.columns)
#     cols.remove('Price')
#     cols.append('Price')
#     data = data[cols]
    
#     rename_cols = {"Levy":"levy", "Manufacturer": "manufacturer",  'Prod. year': "year", 
#               'Category': 'category', 'Leather interior': "leather_interior",'Fuel type':'fuel_type',
#               'Engine volume':'engine_volume', 'Mileage': 'mileage', 'Cylinders': 'cylinders',
#               'Gear box type': 'gear_box_type', 'Drive wheels':'drive_wheels', 'Doors':'doors',
#               "Wheel":'wheel', 'Color':'color', 'Airbags': 'airbags', 'Turbo': "turbo", 'Price': 'price'}
    
#     data = data.rename(columns = rename_cols)
    
#     data.to_csv("data/data.csv")
    
#     return data

#     Fill missing values